# Word2Vec Tutorial

In case you missed the buzz, word2vec is a widely featured as a member of the “new wave” of machine learning algorithms based on neural networks, commonly referred to as "deep learning" (though word2vec itself is rather shallow). Using large amounts of unannotated plain text, word2vec learns relationships between words automatically. The output are vectors, one vector per word, with remarkable linear relationships that allow us to do things like vec(“king”) – vec(“man”) + vec(“woman”) =~ vec(“queen”), or vec(“Montreal Canadiens”) – vec(“Montreal”) + vec(“Toronto”) resembles the vector for “Toronto Maple Leafs”.

Word2vec is very useful in [automatic text tagging](https://github.com/RaRe-Technologies/movie-plots-by-genre), recommender systems and machine translation.

Check out an [online word2vec demo](http://radimrehurek.com/2014/02/word2vec-tutorial/#app) where you can try this vector algebra for yourself. That demo runs `word2vec` on the Google News dataset, of **about 100 billion words**.

## This tutorial

In this tutorial you will learn how to train and evaluate word2vec models on your business data.


## Preparing the Input
Starting from the beginning, gensim’s `word2vec` expects a sequence of sentences as its input. Each sentence is a list of words (utf8 strings):

In [38]:
!pip install gensim
!pip install plotly

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 25.0MB 20kB/s eta 0:00:011 3% |█                               | 849kB 2.2MB/s eta 0:00:12    11% |███▋                            | 2.8MB 2.8MB/s eta 0:00:08    16% |█████▎                          | 4.1MB 2.8MB/s eta 0:00:08    23% |███████▌                        | 5.9MB 3.3MB/s eta 0:00:06    25% |████████                        | 6.2MB 3.2MB/s eta 0:00:06    42% |█████████████▌                  | 10.6MB 2.7MB/s eta 0:00:06    43% |█████████████▉                  | 10.8MB 1.3MB/s eta 0:00:11    44% |██████████████▍                 | 11.2MB 3.3MB/s eta 0:00:05    50% |████████████████                | 12.6MB 1.8MB/s eta 0:00:07    53% |█████████████████▏              | 13.4MB 2.5MB/s eta 0:00:05    58% |██████████████████▊             | 14.6MB 2.5MB/s eta 0:00:05    62% |███████████████████▉      

In [2]:
# import modules & set up logging
import gensim, logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
sentences = [['first', 'sentence'], ['second', 'sentence']]
# train word2vec on the two sentences
model = gensim.models.Word2Vec(sentences, min_count=1)

2018-06-14 14:34:04,010 : INFO : collecting all words and their counts
2018-06-14 14:34:04,016 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-06-14 14:34:04,030 : INFO : collected 3 word types from a corpus of 4 raw words and 2 sentences
2018-06-14 14:34:04,035 : INFO : Loading a fresh vocabulary
2018-06-14 14:34:04,043 : INFO : min_count=1 retains 3 unique words (100% of original 3, drops 0)
2018-06-14 14:34:04,045 : INFO : min_count=1 leaves 4 word corpus (100% of original 4, drops 0)
2018-06-14 14:34:04,047 : INFO : deleting the raw counts dictionary of 3 items
2018-06-14 14:34:04,050 : INFO : sample=0.001 downsamples 3 most-common words
2018-06-14 14:34:04,051 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2018-06-14 14:34:04,052 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2018-06-14 14:34:04,054 : INFO : resetting layer weights
2018-06-14 14:34:04,058 : INFO : training model with 3 workers o

Keeping the input as a Python built-in list is convenient, but can use up a lot of RAM when the input is large.

Gensim only requires that the input must provide sentences sequentially, when iterated over. No need to keep everything in RAM: we can provide one sentence, process it, forget it, load another sentence…

For example, if our input is strewn across several files on disk, with one sentence per line, then instead of loading everything into an in-memory list, we can process the input file by file, line by line:

In [4]:
# create some toy data to use with the following example
import smart_open, os

if not os.path.exists('./data/'):
    os.makedirs('./data/')

filenames = ['./data/f1.txt', './data/f2.txt']

for i, fname in enumerate(filenames):
    with smart_open.smart_open(fname, 'w') as fout:
        for line in sentences[i]:
            fout.write(line + '\n')

In [5]:
from smart_open import smart_open
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in smart_open(os.path.join(self.dirname, fname), 'r'):
                yield line.split()

In [6]:
sentences = MySentences('./data/') # a memory-friendly iterator
print(list(sentences))

[['first'], ['sentence'], ['second'], ['sentence']]


In [7]:
# generate the Word2Vec model
model = gensim.models.Word2Vec(sentences, min_count=1)

2018-06-14 14:34:04,186 : INFO : collecting all words and their counts
2018-06-14 14:34:04,196 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-06-14 14:34:04,212 : INFO : collected 3 word types from a corpus of 4 raw words and 4 sentences
2018-06-14 14:34:04,214 : INFO : Loading a fresh vocabulary
2018-06-14 14:34:04,217 : INFO : min_count=1 retains 3 unique words (100% of original 3, drops 0)
2018-06-14 14:34:04,220 : INFO : min_count=1 leaves 4 word corpus (100% of original 4, drops 0)
2018-06-14 14:34:04,223 : INFO : deleting the raw counts dictionary of 3 items
2018-06-14 14:34:04,226 : INFO : sample=0.001 downsamples 3 most-common words
2018-06-14 14:34:04,229 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2018-06-14 14:34:04,230 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2018-06-14 14:34:04,231 : INFO : resetting layer weights
2018-06-14 14:34:04,232 : INFO : training model with 3 workers o

In [8]:
print(model)
print(model.wv.vocab)

Word2Vec(vocab=3, size=100, alpha=0.025)
{'first': <gensim.models.keyedvectors.Vocab object at 0x7fcfddadcba8>, 'sentence': <gensim.models.keyedvectors.Vocab object at 0x7fcfddadc5f8>, 'second': <gensim.models.keyedvectors.Vocab object at 0x7fcfddadcf60>}


Say we want to further preprocess the words from the files — convert to unicode, lowercase, remove numbers, extract named entities… All of this can be done inside the `MySentences` iterator and `word2vec` doesn’t need to know. All that is required is that the input yields one sentence (list of utf8 words) after another.

**Note to advanced users:** calling `Word2Vec(sentences, iter=1)` will run **two** passes over the sentences iterator. In general it runs `iter+1` passes. By the way, the default value is `iter=5` to comply with Google's word2vec in C language. 
  1. The first pass collects words and their frequencies to build an internal dictionary tree structure. 
  2. The second pass trains the neural model.

These two passes can also be initiated manually, in case your input stream is non-repeatable (you can only afford one pass), and you’re able to initialize the vocabulary some other way:

In [9]:
# build the same model, making the 2 steps explicit
new_model = gensim.models.Word2Vec(min_count=1)  # an empty model, no training
new_model.build_vocab(sentences)                 # can be a non-repeatable, 1-pass generator     
new_model.train(sentences, total_examples=new_model.corpus_count, epochs=new_model.iter)                       
# can be a non-repeatable, 1-pass generator

2018-06-14 14:34:04,395 : INFO : collecting all words and their counts
2018-06-14 14:34:04,406 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-06-14 14:34:04,413 : INFO : collected 3 word types from a corpus of 4 raw words and 4 sentences
2018-06-14 14:34:04,414 : INFO : Loading a fresh vocabulary
2018-06-14 14:34:04,416 : INFO : min_count=1 retains 3 unique words (100% of original 3, drops 0)
2018-06-14 14:34:04,417 : INFO : min_count=1 leaves 4 word corpus (100% of original 4, drops 0)
2018-06-14 14:34:04,418 : INFO : deleting the raw counts dictionary of 3 items
2018-06-14 14:34:04,421 : INFO : sample=0.001 downsamples 3 most-common words
2018-06-14 14:34:04,423 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2018-06-14 14:34:04,427 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2018-06-14 14:34:04,429 : INFO : resetting layer weights
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: 

(1, 20)

In [10]:
print(new_model)
print(model.wv.vocab)

Word2Vec(vocab=3, size=100, alpha=0.025)
{'first': <gensim.models.keyedvectors.Vocab object at 0x7fcfddadcba8>, 'sentence': <gensim.models.keyedvectors.Vocab object at 0x7fcfddadc5f8>, 'second': <gensim.models.keyedvectors.Vocab object at 0x7fcfddadcf60>}


### More data would be nice
For the following examples, we'll use the [Lee Corpus](https://github.com/RaRe-Technologies/gensim/blob/develop/gensim/test/test_data/lee_background.cor) (which you already have if you've installed gensim):

In [11]:
# Set file names for train and test data
test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data']) + os.sep
lee_train_file = test_data_dir + 'lee_background.cor'

In [12]:
class MyText(object):
    def __iter__(self):
        for line in open(lee_train_file):
            # assume there's one document per line, tokens separated by whitespace
            yield line.lower().split()

sentences = MyText()

print(sentences)

## Training
`Word2Vec` accepts several parameters that affect both training speed and quality.

### min_count
`min_count` is for pruning the internal dictionary. Words that appear only once or twice in a billion-word corpus are probably uninteresting typos and garbage. In addition, there’s not enough data to make any meaningful training on those words, so it’s best to ignore them:

In [13]:
# default value of min_count=5
model = gensim.models.Word2Vec(sentences, min_count=10)

2018-06-14 14:34:04,620 : INFO : collecting all words and their counts
2018-06-14 14:34:04,623 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-06-14 14:34:04,648 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2018-06-14 14:34:04,650 : INFO : Loading a fresh vocabulary
2018-06-14 14:34:04,662 : INFO : min_count=10 retains 806 unique words (7% of original 10186, drops 9380)
2018-06-14 14:34:04,664 : INFO : min_count=10 leaves 40964 word corpus (68% of original 59890, drops 18926)
2018-06-14 14:34:04,670 : INFO : deleting the raw counts dictionary of 10186 items
2018-06-14 14:34:04,671 : INFO : sample=0.001 downsamples 54 most-common words
2018-06-14 14:34:04,672 : INFO : downsampling leaves estimated 26224 word corpus (64.0% of prior 40964)
2018-06-14 14:34:04,678 : INFO : estimated required memory for 806 words and 100 dimensions: 1047800 bytes
2018-06-14 14:34:04,680 : INFO : resetting layer weights
2018-06-14 14:3

### size
`size` is the number of dimensions (N) of the N-dimensional space that gensim Word2Vec maps the words onto.

Bigger size values require more training data, but can lead to better (more accurate) models. Reasonable values are in the tens to hundreds.

In [14]:
# default value of size=100
model = gensim.models.Word2Vec(sentences, size=200)

2018-06-14 14:34:04,992 : INFO : collecting all words and their counts
2018-06-14 14:34:04,995 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-06-14 14:34:05,028 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2018-06-14 14:34:05,031 : INFO : Loading a fresh vocabulary
2018-06-14 14:34:05,037 : INFO : min_count=5 retains 1723 unique words (16% of original 10186, drops 8463)
2018-06-14 14:34:05,038 : INFO : min_count=5 leaves 46858 word corpus (78% of original 59890, drops 13032)
2018-06-14 14:34:05,045 : INFO : deleting the raw counts dictionary of 10186 items
2018-06-14 14:34:05,048 : INFO : sample=0.001 downsamples 49 most-common words
2018-06-14 14:34:05,049 : INFO : downsampling leaves estimated 32849 word corpus (70.1% of prior 46858)
2018-06-14 14:34:05,057 : INFO : estimated required memory for 1723 words and 200 dimensions: 3618300 bytes
2018-06-14 14:34:05,059 : INFO : resetting layer weights
2018-06-14 14:

### workers
`workers`, the last of the major parameters (full list [here](http://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec)) is for training parallelization, to speed up training:

In [15]:
# default value of workers=3 (tutorial says 1...)
model = gensim.models.Word2Vec(sentences, workers=4)

2018-06-14 14:34:05,556 : INFO : collecting all words and their counts
2018-06-14 14:34:05,559 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-06-14 14:34:05,584 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2018-06-14 14:34:05,594 : INFO : Loading a fresh vocabulary
2018-06-14 14:34:05,605 : INFO : min_count=5 retains 1723 unique words (16% of original 10186, drops 8463)
2018-06-14 14:34:05,610 : INFO : min_count=5 leaves 46858 word corpus (78% of original 59890, drops 13032)
2018-06-14 14:34:05,618 : INFO : deleting the raw counts dictionary of 10186 items
2018-06-14 14:34:05,619 : INFO : sample=0.001 downsamples 49 most-common words
2018-06-14 14:34:05,620 : INFO : downsampling leaves estimated 32849 word corpus (70.1% of prior 46858)
2018-06-14 14:34:05,627 : INFO : estimated required memory for 1723 words and 100 dimensions: 2239900 bytes
2018-06-14 14:34:05,628 : INFO : resetting layer weights
2018-06-14 14:

The `workers` parameter only has an effect if you have [Cython](http://cython.org/) installed. Without Cython, you’ll only be able to use one core because of the [GIL](https://wiki.python.org/moin/GlobalInterpreterLock) (and `word2vec` training will be [miserably slow](http://rare-technologies.com/word2vec-in-python-part-two-optimizing/)).

## Memory
At its core, `word2vec` model parameters are stored as matrices (NumPy arrays). Each array is **#vocabulary** (controlled by min_count parameter) times **#size** (size parameter) of floats (single precision aka 4 bytes).

Three such matrices are held in RAM (work is underway to reduce that number to two, or even one). So if your input contains 100,000 unique words, and you asked for layer `size=200`, the model will require approx. `100,000*200*4*3 bytes = ~229MB`.

There’s a little extra memory needed for storing the vocabulary tree (100,000 words would take a few megabytes), but unless your words are extremely loooong strings, memory footprint will be dominated by the three matrices above.

## Evaluating
`Word2Vec` training is an unsupervised task, there’s no good way to objectively evaluate the result. Evaluation depends on your end application.

Google has released their testing set of about 20,000 syntactic and semantic test examples, following the “A is to B as C is to D” task. It is provided in the 'datasets' folder.

For example a syntactic analogy of comparative type is bad:worse;good:?. There are total of 9 types of syntactic comparisons in the dataset like plural nouns and nouns of opposite meaning.

The semantic questions contain five types of semantic analogies, such as capital cities (Paris:France;Tokyo:?) or family members (brother:sister;dad:?). 

Gensim supports the same evaluation set, in exactly the same format:

In [16]:
model.accuracy('./datasets/questions-words.txt')

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `accuracy` (Method will be removed in 4.0.0, use self.wv.accuracy() instead).
  """Entry point for launching an IPython kernel.
2018-06-14 14:34:06,051 : INFO : precomputing L2-norms of word weight vectors
2018-06-14 14:34:06,057 : INFO : family: 0.0% (0/2)
2018-06-14 14:34:06,077 : INFO : gram3-comparative: 0.0% (0/12)
2018-06-14 14:34:06,088 : INFO : gram4-superlative: 0.0% (0/12)
2018-06-14 14:34:06,103 : INFO : gram5-present-participle: 0.0% (0/20)
2018-06-14 14:34:06,121 : INFO : gram6-nationality-adjective: 0.0% (0/20)
2018-06-14 14:34:06,135 : INFO : gram7-past-tense: 0.0% (0/20)
2018-06-14 14:34:06,147 : INFO : gram8-plural: 0.0% (0/12)
2018-06-14 14:34:06,155 : INFO : total: 0.0% (0/98)


[{'correct': [], 'incorrect': [], 'section': 'capital-common-countries'},
 {'correct': [], 'incorrect': [], 'section': 'capital-world'},
 {'correct': [], 'incorrect': [], 'section': 'currency'},
 {'correct': [], 'incorrect': [], 'section': 'city-in-state'},
 {'correct': [],
  'incorrect': [('HE', 'SHE', 'HIS', 'HER'), ('HIS', 'HER', 'HE', 'SHE')],
  'section': 'family'},
 {'correct': [], 'incorrect': [], 'section': 'gram1-adjective-to-adverb'},
 {'correct': [], 'incorrect': [], 'section': 'gram2-opposite'},
 {'correct': [],
  'incorrect': [('GOOD', 'BETTER', 'GREAT', 'GREATER'),
   ('GOOD', 'BETTER', 'LONG', 'LONGER'),
   ('GOOD', 'BETTER', 'LOW', 'LOWER'),
   ('GREAT', 'GREATER', 'LONG', 'LONGER'),
   ('GREAT', 'GREATER', 'LOW', 'LOWER'),
   ('GREAT', 'GREATER', 'GOOD', 'BETTER'),
   ('LONG', 'LONGER', 'LOW', 'LOWER'),
   ('LONG', 'LONGER', 'GOOD', 'BETTER'),
   ('LONG', 'LONGER', 'GREAT', 'GREATER'),
   ('LOW', 'LOWER', 'GOOD', 'BETTER'),
   ('LOW', 'LOWER', 'GREAT', 'GREATER'),
   (

This `accuracy` takes an 
[optional parameter](http://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec.accuracy) `restrict_vocab` 
which limits which test examples are to be considered.



In the December 2016 release of Gensim we added a better way to evaluate semantic similarity.

By default it uses an academic dataset WS-353 but one can create a dataset specific to your business based on it. It contains word pairs together with human-assigned similarity judgments. It measures the relatedness or co-occurrence of two words. For example, 'coast' and 'shore' are very similar as they appear in the same context. At the same time 'clothes' and 'closet' are less similar because they are related but not interchangeable.

In [17]:
model.evaluate_word_pairs(test_data_dir + 'wordsim353.tsv')

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `evaluate_word_pairs` (Method will be removed in 4.0.0, use self.wv.evaluate_word_pairs() instead).
  """Entry point for launching an IPython kernel.
2018-06-14 14:34:06,201 : INFO : Pearson correlation coefficient against /opt/conda/lib/python3.6/site-packages/gensim/test/test_data/wordsim353.tsv: 0.0993
2018-06-14 14:34:06,203 : INFO : Spearman rank-order correlation coefficient against /opt/conda/lib/python3.6/site-packages/gensim/test/test_data/wordsim353.tsv: 0.0623
2018-06-14 14:34:06,204 : INFO : Pairs with unknown words ratio: 85.6%


((0.099326693481591385, 0.48801650009644326),
 SpearmanrResult(correlation=0.062269487642049343, pvalue=0.66422285754165034),
 85.55240793201133)

Once again, **good performance on Google's or WS-353 test set doesn’t mean word2vec will work well in your application, or vice versa**. It’s always best to evaluate directly on your intended task. For an example of how to use word2vec in a classifier pipeline, see this [tutorial](https://github.com/RaRe-Technologies/movie-plots-by-genre).

## Storing and loading models
You can store/load models using the standard gensim methods:

In [18]:
from tempfile import mkstemp

fs, temp_path = mkstemp("gensim_temp")  # creates a temp file

model.save(temp_path)  # save the model

2018-06-14 14:34:06,224 : INFO : saving Word2Vec object under /tmp/tmpb8tbvqehgensim_temp, separately None
2018-06-14 14:34:06,226 : INFO : not storing attribute vectors_norm
2018-06-14 14:34:06,229 : INFO : not storing attribute cum_table
2018-06-14 14:34:06,246 : INFO : saved /tmp/tmpb8tbvqehgensim_temp


In [19]:
new_model = gensim.models.Word2Vec.load(temp_path)  # open the model

2018-06-14 14:34:06,260 : INFO : loading Word2Vec object from /tmp/tmpb8tbvqehgensim_temp
2018-06-14 14:34:06,276 : INFO : loading wv recursively from /tmp/tmpb8tbvqehgensim_temp.wv.* with mmap=None
2018-06-14 14:34:06,277 : INFO : setting ignored attribute vectors_norm to None
2018-06-14 14:34:06,279 : INFO : loading vocabulary recursively from /tmp/tmpb8tbvqehgensim_temp.vocabulary.* with mmap=None
2018-06-14 14:34:06,280 : INFO : loading trainables recursively from /tmp/tmpb8tbvqehgensim_temp.trainables.* with mmap=None
2018-06-14 14:34:06,281 : INFO : setting ignored attribute cum_table to None
2018-06-14 14:34:06,282 : INFO : loaded /tmp/tmpb8tbvqehgensim_temp


which uses pickle internally, optionally `mmap`‘ing the model’s internal large NumPy matrices into virtual memory directly from disk files, for inter-process memory sharing.

In addition, you can load models created by the original C tool, both using its text and binary formats:
```
  model = gensim.models.KeyedVectors.load_word2vec_format('/tmp/vectors.txt', binary=False)
  # using gzipped/bz2 input works too, no need to unzip:
  model = gensim.models.KeyedVectors.load_word2vec_format('/tmp/vectors.bin.gz', binary=True)
```

## Online training / Resuming training
Advanced users can load a model and continue training it with more sentences and [new vocabulary words](online_w2v_tutorial.ipynb):

In [20]:
model = gensim.models.Word2Vec.load(temp_path)
more_sentences = [['Advanced', 'users', 'can', 'load', 'a', 'model', 'and', 'continue', 'training', 'it', 'with', 'more', 'sentences']]
model.build_vocab(more_sentences, update=True)
model.train(more_sentences, total_examples=model.corpus_count, epochs=model.iter)

# cleaning up temp
os.close(fs)
os.remove(temp_path)

2018-06-14 14:34:06,304 : INFO : loading Word2Vec object from /tmp/tmpb8tbvqehgensim_temp
2018-06-14 14:34:06,320 : INFO : loading wv recursively from /tmp/tmpb8tbvqehgensim_temp.wv.* with mmap=None
2018-06-14 14:34:06,321 : INFO : setting ignored attribute vectors_norm to None
2018-06-14 14:34:06,322 : INFO : loading vocabulary recursively from /tmp/tmpb8tbvqehgensim_temp.vocabulary.* with mmap=None
2018-06-14 14:34:06,323 : INFO : loading trainables recursively from /tmp/tmpb8tbvqehgensim_temp.trainables.* with mmap=None
2018-06-14 14:34:06,324 : INFO : setting ignored attribute cum_table to None
2018-06-14 14:34:06,326 : INFO : loaded /tmp/tmpb8tbvqehgensim_temp
2018-06-14 14:34:06,331 : INFO : collecting all words and their counts
2018-06-14 14:34:06,332 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-06-14 14:34:06,333 : INFO : collected 13 word types from a corpus of 13 raw words and 1 sentences
2018-06-14 14:34:06,335 : INFO : Updating model with 

You may need to tweak the `total_words` parameter to `train()`, depending on what learning rate decay you want to simulate.

Note that it’s not possible to resume training with models generated by the C tool, `KeyedVectors.load_word2vec_format()`. You can still use them for querying/similarity, but information vital for training (the vocab tree) is missing there.

## Using the model
`Word2Vec` supports several word similarity tasks out of the box:

In [21]:
model.most_similar(positive=['human', 'crime'], negative=['party'], topn=1)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
2018-06-14 14:34:06,427 : INFO : precomputing L2-norms of word weight vectors


[('wake', 0.995140552520752)]

In [22]:
model.doesnt_match("input is lunch he sentence cat".split())

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
2018-06-14 14:34:06,439 : WARNING : vectors for words {'lunch', 'cat', 'input'} are not present in the model, ignoring these words


'sentence'

In [23]:
print(model.similarity('human', 'party'))
print(model.similarity('tree', 'murder'))

0.999116843916
0.993565185153


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


You can get the probability distribution for the center word given the context words as input:

In [24]:
print(model.predict_output_word(['emergency', 'beacon', 'received']))

[('more', 0.00094383128), ('government', 0.00077048101), ('training', 0.0007607498), ('continue', 0.00075521087), ('can', 0.00074793148), ('australia', 0.00074786448), ('also', 0.00074378337), ('united', 0.0007307519), ('there', 0.00072391168), ('arafat', 0.00072192884)]


The results here don't look good because the training corpus is very small. To get meaningful results one needs to train on 500k+ words.

If you need the raw output vectors in your application, you can access these either on a word-by-word basis:

In [25]:
model['tree']  # raw NumPy vector of a word

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.00507236,  0.01695369, -0.04579246,  0.04015445, -0.03821801,
       -0.00806551,  0.04910685, -0.03377812, -0.06309545,  0.03624553,
        0.01632687,  0.00671562, -0.02742973, -0.01133134,  0.0035804 ,
        0.01326715,  0.0168173 , -0.02178709,  0.03465009,  0.0069912 ,
        0.02599325,  0.00419654, -0.01365976, -0.0308643 , -0.02550944,
        0.00818279,  0.01674226, -0.03719213,  0.04906338,  0.02477256,
        0.08780177,  0.02048765,  0.01182354,  0.04090331,  0.02517812,
       -0.0316829 , -0.02182687, -0.06434128, -0.04797867, -0.01253023,
       -0.02586541,  0.01935347, -0.04413845,  0.02180763, -0.07296643,
        0.01786931, -0.050268  ,  0.00752149, -0.03242191, -0.01589916,
       -0.01787307,  0.01419847, -0.03283262, -0.011434  , -0.05323173,
        0.03757442, -0.02009087,  0.01703732,  0.03100888, -0.02094323,
       -0.00066713,  0.02878382, -0.03230232,  0.02279637,  0.04172381,
        0.01062667,  0.03786817,  0.08861102,  0.02583497,  0.05

…or en-masse as a 2D NumPy matrix from `model.wv.syn0`.

## Training Loss Computation

The parameter `compute_loss` can be used to toggle computation of loss while training the Word2Vec model. The computed loss is stored in the model attribute `running_training_loss` and can be retrieved using the function `get_latest_training_loss` as follows : 

In [26]:
# instantiating and training the Word2Vec model
model_with_loss = gensim.models.Word2Vec(sentences, min_count=1, compute_loss=True, hs=0, sg=1, seed=42)

# getting the training loss value
training_loss = model_with_loss.get_latest_training_loss()
print(training_loss)

2018-06-14 14:34:06,503 : INFO : collecting all words and their counts
2018-06-14 14:34:06,506 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-06-14 14:34:06,545 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2018-06-14 14:34:06,556 : INFO : Loading a fresh vocabulary
2018-06-14 14:34:06,597 : INFO : min_count=1 retains 10186 unique words (100% of original 10186, drops 0)
2018-06-14 14:34:06,599 : INFO : min_count=1 leaves 59890 word corpus (100% of original 59890, drops 0)
2018-06-14 14:34:06,630 : INFO : deleting the raw counts dictionary of 10186 items
2018-06-14 14:34:06,631 : INFO : sample=0.001 downsamples 37 most-common words
2018-06-14 14:34:06,632 : INFO : downsampling leaves estimated 47231 word corpus (78.9% of prior 59890)
2018-06-14 14:34:06,662 : INFO : estimated required memory for 10186 words and 100 dimensions: 13241800 bytes
2018-06-14 14:34:06,664 : INFO : resetting layer weights
2018-06-14 14:34

1814180.75


### Benchmarks to see effect of training loss compuation code on training time

We first download and setup the test data used for getting the benchmarks.

In [27]:
def get_data_lee_background_file():
    test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data'])
    lee_train_file = test_data_dir + os.sep + 'lee_background.cor'
    return lee_train_file

In [28]:
input_data_files = []

def setup_input_data():
    # check if test data already present
    if os.path.isfile('./text8') is False:

        # download and decompress 'text8' corpus
        import zipfile
        ! wget 'http://mattmahoney.net/dc/text8.zip'
        ! unzip 'text8.zip'
    
        # create 1 MB, 10 MB and 50 MB files
        ! head -c1000000 text8 > text8_1000000
        ! head -c10000000 text8 > text8_10000000
        ! head -c50000000 text8 > text8_50000000
                
    # add 25 KB test file
#     input_data_files.append(os.path.join(os.getcwd(), '../../gensim/test/test_data/lee_background.cor'))
    input_data_files.append(get_data_lee_background_file())

    # add 1 MB test file
    input_data_files.append(os.path.join(os.getcwd(), 'text8_1000000'))

    # add 10 MB test file
    input_data_files.append(os.path.join(os.getcwd(), 'text8_10000000'))

    # add 50 MB test file
    input_data_files.append(os.path.join(os.getcwd(), 'text8_50000000'))

    # add 100 MB test file
    input_data_files.append(os.path.join(os.getcwd(), 'text8'))

setup_input_data()
print(input_data_files)

['/opt/conda/lib/python3.6/site-packages/gensim/test/test_data/lee_background.cor', '/home/jovyan/Notebooks/text8_1000000', '/home/jovyan/Notebooks/text8_10000000', '/home/jovyan/Notebooks/text8_50000000', '/home/jovyan/Notebooks/text8']


We now compare the training time taken for different combinations of input data and model training parameters like `hs` and `sg`.

In [29]:
logging.getLogger().setLevel(logging.ERROR)

In [30]:
print(get_data_lee_background_file())
print(input_data_files[0])

/opt/conda/lib/python3.6/site-packages/gensim/test/test_data/lee_background.cor
/opt/conda/lib/python3.6/site-packages/gensim/test/test_data/lee_background.cor


In [31]:
# using 25 KB and 50 MB files only for generating o/p -> comment next line for using all 5 test files
input_data_files = [input_data_files[0], input_data_files[-2]]
print(input_data_files)

import time
import numpy as np
import pandas as pd

train_time_values = []
seed_val = 42
sg_values = [0, 1]
hs_values = [0, 1]

for data_file in input_data_files:
    data = gensim.models.word2vec.LineSentence(data_file) 
    for sg_val in sg_values:
        for hs_val in hs_values:
            for loss_flag in [True, False]:
                time_taken_list = []
                for i in range(3):
                    start_time = time.time()
                    w2v_model = gensim.models.Word2Vec(data, compute_loss=loss_flag, sg=sg_val, hs=hs_val, seed=seed_val) 
                    time_taken_list.append(time.time() - start_time)

                time_taken_list = np.array(time_taken_list)
                time_mean = np.mean(time_taken_list)
                time_std = np.std(time_taken_list)
                train_time_values.append({'train_data': data_file, 'compute_loss': loss_flag, 'sg': sg_val, 'hs': hs_val, 'mean': time_mean, 'std': time_std})

train_times_table = pd.DataFrame(train_time_values)
train_times_table = train_times_table.sort_values(by=['train_data', 'sg', 'hs', 'compute_loss'], ascending=[False, False, True, False])
print(train_times_table)

['/opt/conda/lib/python3.6/site-packages/gensim/test/test_data/lee_background.cor', '/home/jovyan/Notebooks/text8_50000000']
   compute_loss  hs        mean  sg         std  \
4          True   0    0.762173   1    0.026214   
5         False   0    0.832185   1    0.052484   
6          True   1    1.630927   1    0.100058   
7         False   1    1.664473   1    0.031450   
0          True   0    0.372066   0    0.025527   
1         False   0    0.328010   0    0.023712   
2          True   1    0.553692   0    0.017070   
3         False   1    0.532775   0    0.018570   
12         True   0  162.599266   1    3.844931   
13        False   0  159.110715   1    2.070048   
14         True   1  321.043705   1    3.589240   
15        False   1  314.815207   1    0.619422   
8          True   0   59.193477   0    5.292913   
9         False   0   64.865958   0    7.508053   
10         True   1  604.389132   0  669.734041   
11        False   1  110.832784   0   18.025254   

       

### Adding Word2Vec "model to dict" method to production pipeline
Suppose, we still want more performance improvement in production. 
One good way is to cache all the similar words in a dictionary.
So that next time when we get the similar query word, we'll search it first in the dict.
And if it's a hit then we will show the result directly from the dictionary.
otherwise we will query the word and then cache it so that it doesn't miss next time.

In [32]:
FORMAT = '%(asctime)s : %(levelname)s : %(message)s'
logging.basicConfig(format=FORMAT)
logging.getLogger().setLevel(level=logging.INFO)

# logging.getLogger().setLevel(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [33]:
most_similars_precalc = {word : model.wv.most_similar(word) for word in model.wv.index2word}
for i, (key, value) in enumerate(most_similars_precalc.items()):
    if i==3:
        break
    print (key, value)

the [('australian', 0.9999346733093262), ('a', 0.9999293088912964), ('up', 0.9999282360076904), ('its', 0.9999280571937561), ('and', 0.9999231100082397), ('two', 0.9999219179153442), ('an', 0.9999213814735413), ('by', 0.9999197721481323), ('their', 0.9999196529388428), ('first', 0.9999193549156189)]
to [('but', 0.999959409236908), ('will', 0.9999560713768005), ('are', 0.9999557733535767), ('from', 0.9999555349349976), ('would', 0.9999527931213379), ('on', 0.9999493956565857), ('as', 0.9999487400054932), ('his', 0.9999485015869141), ('and', 0.9999483823776245), ('last', 0.9999483823776245)]
of [('at', 0.9999558329582214), ('in', 0.9999525547027588), ('were', 0.9999501705169678), ('into', 0.9999499917030334), ('after', 0.9999499320983887), ('with', 0.99994957447052), ('and', 0.9999489784240723), ('by', 0.9999462962150574), ('from', 0.9999452829360962), ('has', 0.9999441504478455)]


### Comparison with and without caching

for time being lets take 4 words randomly

In [34]:
import time
words = ['voted','few','their','around']

Without caching

In [35]:
start = time.time()
for word in words:
    result = model.wv.most_similar(word)
    print(result)
end = time.time()
print(end-start)

[('rates', 0.998995840549469), ('we', 0.9989607334136963), ('forces', 0.9989598393440247), ('jihad', 0.998958945274353), ('should', 0.9989583492279053), ('them', 0.9989446997642517), ('but', 0.9989413619041443), ('few', 0.9989379644393921), ('any', 0.9989374876022339), ('"the', 0.99893718957901)]
[('are', 0.9997949600219727), ('last', 0.9997931122779846), ('have', 0.9997919797897339), ('up', 0.9997895359992981), ('against', 0.9997879862785339), ('from', 0.9997867345809937), ('one', 0.9997856020927429), ('has', 0.9997854232788086), ('and', 0.9997849464416504), ('world', 0.9997841119766235)]
[('and', 0.9999523758888245), ('are', 0.9999518394470215), ('up', 0.9999505281448364), ('with', 0.9999479651451111), ('this', 0.999947190284729), ('on', 0.9999443292617798), ('had', 0.9999441504478455), ('by', 0.999943196773529), ('as', 0.9999431371688843), ('which', 0.9999417066574097)]
[('people', 0.9999455213546753), ('and', 0.9999428987503052), ('after', 0.9999410510063171), ('from', 0.9999408721

Now with caching

In [36]:
start = time.time()
for word in words:
    if 'voted' in most_similars_precalc:
        result = most_similars_precalc[word]
        print(result)
    else:
        result = model.wv.most_similar(word)
        most_similars_precalc[word] = result
        print(result)
    
end = time.time()
print(end-start)

[('rates', 0.998995840549469), ('we', 0.9989607334136963), ('forces', 0.9989598393440247), ('jihad', 0.998958945274353), ('should', 0.9989583492279053), ('them', 0.9989446997642517), ('but', 0.9989413619041443), ('few', 0.9989379644393921), ('any', 0.9989374876022339), ('"the', 0.99893718957901)]
[('are', 0.9997949600219727), ('last', 0.9997931122779846), ('have', 0.9997919797897339), ('up', 0.9997895359992981), ('against', 0.9997879862785339), ('from', 0.9997867345809937), ('one', 0.9997856020927429), ('has', 0.9997854232788086), ('and', 0.9997849464416504), ('world', 0.9997841119766235)]
[('and', 0.9999523758888245), ('are', 0.9999518394470215), ('up', 0.9999505281448364), ('with', 0.9999479651451111), ('this', 0.999947190284729), ('on', 0.9999443292617798), ('had', 0.9999441504478455), ('by', 0.999943196773529), ('as', 0.9999431371688843), ('which', 0.9999417066574097)]
[('people', 0.9999455213546753), ('and', 0.9999428987503052), ('after', 0.9999410510063171), ('from', 0.9999408721

Clearly you can see the improvement but this difference will be even larger when we take more words in the consideration.

# Visualising the Word Embeddings

The word embeddings made by the model can be visualised by reducing dimensionality of the words to 2 dimensions using tSNE.

Visualisations can be used to notice semantic and syntactic trends in the data.

Example: Semantic- words like cat, dog, cow, etc. have a tendency to lie close by
         Syntactic- words like run, running or cut, cutting lie close together.
Vector relations like vKing - vMan = vQueen - vWoman can also be noticed.

Additional dependencies : 
- sklearn
- numpy
- plotly

The function below can be used to plot the embeddings in an ipython notebook.
It requires the model as the necessary parameter. If you don't have the model, you can load it by

`model = gensim.models.Word2Vec.load('path/to/model')`

If you don't want to plot inside a notebook, set the `plot_in_notebook` parameter to `False`.

Note: the model used for the visualisation is trained on a small corpus. Thus some of the relations might not be so clear

Beware : This sort dimension reduction comes at the cost of loss of information.

In [39]:
from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction
import numpy as np                                  # array handling

from plotly.offline import init_notebook_mode, iplot, plot
import plotly.graph_objs as go

def reduce_dimensions(model, plot_in_notebook = True):

    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    vectors = []        # positions in vector space
    labels = []         # keep track of words to label our data again later
    for word in model.wv.vocab:
        vectors.append(model[word])
        labels.append(word)


    # convert both lists into numpy vectors for reduction
    vectors = np.asarray(vectors)
    labels = np.asarray(labels)
    
    # reduce using t-SNE
    vectors = np.asarray(vectors)
    logging.info('starting tSNE dimensionality reduction. This may take some time.')
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
        
    # Create a trace
    trace = go.Scatter(
        x=x_vals,
        y=y_vals,
        mode='text',
        text=labels
        )
    
    data = [trace]
    
    logging.info('All done. Plotting.')
    
    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')

In [40]:
reduce_dimensions(model)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

2018-06-14 23:33:25,046 : INFO : starting tSNE dimensionality reduction. This may take some time.
2018-06-14 23:33:36,806 : INFO : All done. Plotting.


## Conclusion

In this tutorial we learned how to train word2vec models on your custom data and also how to evaluate it. Hope that you too will find this popular tool useful in your Machine Learning tasks!

## Links


Full `word2vec` API docs [here](http://radimrehurek.com/gensim/models/word2vec.html); get [gensim](http://radimrehurek.com/gensim/) here. Original C toolkit and `word2vec` papers by Google [here](https://code.google.com/archive/p/word2vec/).